**Importing necessary libraries:**

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
import pandas
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
from google.colab import drive
from tensorflow.keras import regularizers

**Natural Language Toolkit:**

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data_reddit = pandas.read_csv("Reddit.csv")
data_twitter = pandas.read_csv("Twitter.csv")
data_reddit.rename(columns = {'clean_comment': 'text'}, inplace = True)
data_twitter.rename(columns = {'clean_text': 'text'}, inplace = True)

data = pandas.concat([data_reddit, data_twitter], ignore_index = True)
data = pandas.DataFrame(data)
data.dropna(axis = 0, inplace = True)

**Preprocessing Text**

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):

        text = text.lower()

        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token.isalnum()]

        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text
    else:
        return ""

def tokenize(item):
  tokenizer = Tokenizer(oov_token="")
  tokenizer.fit_on_texts(item)
  sequences = tokenizer.texts_to_sequences(item)
  max_length = 65
  padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
  vocab_size = len(tokenizer.word_index) + 1
  return padded_sequences, vocab_size

def lr_schedule(epoch, lr):
    return lr * np.exp(1)

In [ ]:
ordinal_mapping = {-1: 0, 0: 1, 1: 2}


data['encoded_sentiment'] = data['category'].apply(lambda x: ordinal_mapping[x])
data['preprocessed_text'] = data['text'].apply(preprocess_text)

**Splitting training and test data:**

In [ ]:
x = data.pop("preprocessed_text")
y = data.pop('encoded_sentiment')
x, vocab_size = tokenize(x)
y = to_categorical(y, 3)

X_train,X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=0, verbose=1, min_lr=1e-6)

embedding_dim = 65
max_length = 65

**Training the model using LSTM:**

In [ ]:
tf.random.set_seed(42)
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(8, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))
model.add(LSTM(8, return_sequences=True))
model.add(Dropout(rate=0.4))
model.add(LSTM(8, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(8))
model.add(Dropout(rate=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.004), metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=25, validation_split=0.2, callbacks=[early_stopping,lr_scheduler])

Epoch 1/25
4253/4253 [==============================] - 130s 28ms/step - loss: 0.4971 - accuracy: 0.8304 - val_loss: 0.3699 - val_accuracy: 0.8869 - lr: 0.0040
Epoch 2/25
4253/4253 [==============================] - ETA: 0s - loss: 0.3680 - accuracy: 0.8947
Epoch 2: ReduceLROnPlateau reducing learning rate to 0.002400000113993883.
4253/4253 [==============================] - 82s 19ms/step - loss: 0.3680 - accuracy: 0.8947 - val_loss: 0.3789 - val_accuracy: 0.8907 - lr: 0.0040
Epoch 3/25
4253/4253 [==============================] - 86s 20ms/step - loss: 0.3085 - accuracy: 0.9167 - val_loss: 0.3111 - val_accuracy: 0.9100 - lr: 0.0024
Epoch 4/25
4253/4253 [==============================] - ETA: 0s - loss: 0.2658 - accuracy: 0.9283
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00144000006839633.
4253/4253 [==============================] - 79s 18ms/step - loss: 0.2658 - accuracy: 0.9283 - val_loss: 0.3251 - val_accuracy: 0.9061 - lr: 0.0024
Epoch 5/25
4253/4253 [==================

**Testing the model:**

In [ ]:
model.evaluate(X_test,y_test)

939/939 [==============================] - 6s 6ms/step - loss: 0.3199 - accuracy: 0.9082


[0.3199221193790436, 0.9082217216491699]